# Introduksjon til PySCF

**PySCF (Python-based Simulations of Chemistry Framework)** er en åpen kildekode-pakke for kvantekjemi som er utviklet for å utføre en ulike elektronstrukturteoriberegninger (langt ord!). Det brukes mye til både forskning og undervisning på grunn av sin fleksibilitet, effektivitet og brukervennlighet. Navnet er et ordspill – "SCF" står vanligvis for Self-Consistent Field, altså Hartree-Fock og Tetthetsfunksjonalteori (DFT). Disse metodene er blant PySCFs sterkeste sider.

Dokumentasjonen er god, og et godt sted å starte er den offisielle nettsiden: https://pyscf.org/index.html


### Noen nøkkelfunksjoner i PySCF:
1. **Kjernefunksjonaliteter**:
   - PySCF støtter **Hartree-Fock (HF)**- og **Tetthetsfunksjonalteori (DFT)**-beregninger, samt avanserte metoder etter Hartree-Fock som **Møller-Plesset pertubasjonsteori (MP2)**, **Coupled Cluster (CC)**, **Konfigurasjonsinteraksjon (CI)**, **Complete Active Space Self-Consistent Field (CASSCF)**-beregninger og flere.

2. **Basis-sett og integraler**:
   - Programvaren inkluderer en rekke **Gaussiske orbitaler (GTO)** basis-sett og effektive rutiner for å beregne integraler.

3. **Modularitet og fleksibilitet**:
   - PySCF er designet for å være modulær, noe som gjør det mulig for brukere å utvide funksjonaliteten eller integrere den med andre verktøy. Dette gjør den egnet for skreddersydde arbeidsflyter i forskning.

4. **Python-grensesnitt**:
   - Skrevet i Python med ytelseskritiske deler i C, tilbyr PySCF et intuitivt og brukervennlig grensesnitt som utnytter Python-økosystemet for skripting og rask utvikling.
  

# En første beregning med Hartree-Fock

I denne delen av notebooken skal vi se hvordan vi definerer et molekyl, velger et GTO-basissett, og kjører en Hartree-Fock-beregning på molekylet.

## Importering av relevante pakker

In [1]:
from pyscf import * 
import matplotlib.pyplot as plt

## Definere et molekyl

Et molekyl defineres ved å plassere kjerner på posisjoner i rommet gitt ved XYZ-koordinater. PySCF regner ut hvor mange elektroner molekylet har under antagelsen at det er elektrisk nøytralt. Her definerer vi et vannmolekyl (med en litt rar geometri), og - for moro skyld - koffein-molekylet (bør være nær likevekt).

Når geometrien er bestemt, må man så velge et AO basissett. Husk at AO står for "atomic orbital". Her velger vi et basissett som heter STO-3G. Dette er en såkalt "minimal basis", der hvert elektron starter med 1 orbital, sentrert på atomet elektronet "hører til". I dette eksempelet vil det derfor være 1 basisfunksjon for hvert H-atom, og 5 basisfunksjoner for O-atomet. Hver basisfunksjon er en Slater-type orbital som er tilnærmet med 3 gaussiske orbitaler.

Så må vi be PySCF å "bygge" molekylet for å få en veldefinert datastruktur som PySCF kan gjøre beregninger med.

For mer informasjon og flere måter å gjøre dette på, se https://pyscf.org/user/gto.html.



In [2]:
water = """
O          0.00000        0.00000        0.00000
H          0.27740        0.89290        0.25440
H          0.60680       -0.23830       -0.71690

"""

caffeine = """
O          0.47000        2.56880        0.00060
O         -3.12710       -0.44360       -0.00030
N         -0.96860       -1.31250        0.00000
N          2.21820        0.14120       -0.00030
N         -1.34770        1.07970       -0.00010
N          1.41190       -1.93720        0.00020
C          0.85790        0.25920       -0.00080
C          0.38970       -1.02640       -0.00040
C          0.03070        1.42200       -0.00060
C         -1.90610       -0.24950       -0.00040
C          2.50320       -1.19980        0.00030
C         -1.42760       -2.69600        0.00080
C          3.19260        1.20610        0.00030
C         -2.29690        2.18810        0.00070
H          3.51630       -1.57870        0.00080
H         -1.04510       -3.19730       -0.89370
H         -2.51860       -2.75960        0.00110
H         -1.04470       -3.19630        0.89570
H          4.19920        0.78010        0.00020
H          3.04680        1.80920       -0.89920
H          3.04660        1.80830        0.90040
H         -1.80870        3.16510       -0.00030
H         -2.93220        2.10270        0.88810
H         -2.93460        2.10210       -0.88490
"""


mol = gto.Mole()
mol.atom = water
mol.basis = 'cc-PVTZ'
mol.unit = 'Angstrom' # Alternativt 'Bohr'
mol.build()



## Kjøre en Hartree-Fock-beregning

Når vi har bestemt molekylet vårt, kan vi gjøre, for eksempel, en HF-beregning. Dette gjøres i to steg:
1. Lage et `RHF`-objekt som lar oss gjøre en Restricted Hartree-Fock-beregning. Dette objektet initialiseres med `Mole`-objektet fra tidligere.
2. Gjøre beregningene.



In [3]:
mf = scf.RHF(mol)  # Set up a Restricted Hartree-Fock solver
energy = mf.kernel()   # Solve for the energy

print("Energy: ", energy)

converged SCF energy = -76.0560886658238
Energy:  -76.05608866582375


## Valg av basissett

Kvantekjemi inneholder et veritabelt overflødighetshorn av GTO-baserte basissett, tilpasset mange ulike formål. I eksempelet over ble basissettet `STO-3G` brukt. Her er en tabell med flere basissett å prøve ut:

| **Basissett**    | **Type**        | **Beskrivelse**                                                                                                      | **Typiske bruksområder**                   |
|------------------|-----------------|----------------------------------------------------------------------------------------------------------------------|--------------------------------------------|
| **STO-3G**       | Minimalt Basis   | Bruker 3 Gauss-funksjoner for å tilnærme én Slater-type orbital. Rask, men med minimal nøyaktighet.                   | Små molekyler, enkle beregninger           |
| **3-21G**        | Split-Valence    | Splitter valensorbitalene i to Gauss-funksjoner for større fleksibilitet.                                             | Små systemer, moderat nøyaktighet          |
| **6-31G**        | Split-Valence    | Mer nøyaktig enn 3-21G; bruker 6 Gauss-funksjoner for kjernorbitaler og en delt valensorbitaltilnærming.              | Organiske molekyler, geometrioptimalisering|
| **cc-pVDZ**      | Korrelasjons-Konsistent | Dobbelt-zeta basissett designet for korrelerte bølgefunksjonsmetoder som CCSD og MP2.                               | Korrelerte beregninger, små molekyler      |
| **cc-pVTZ**      | Korrelasjons-Konsistent | Trippel-zeta versjon for bedre nøyaktighet, spesielt for korrelerte metoder.                                         | Nøyaktige korrelerte metoder               |
| **cc-pVQZ**      | Korrelasjons-Konsistent | Kvadrupel-zeta versjon for svært høy nøyaktighet i korrelerte beregninger.                                           | Høynøyaktige beregninger, referanseverdier |
| **def2-SVP**     | Split-Valence    | Dobbelt-zeta med polarisasjonsfunksjoner, optimalisert for DFT og korrelerte metoder.                                | Middels store molekyler, geometrioptimalisering |
| **def2-TZVP**    | Trippel-Zeta     | Trippel-zeta basissett med polarisasjon. Høy nøyaktighet, egnet for større systemer.                                  | DFT og bølgefunksjonsbaserte metoder       |
| **def2-QZVP**    | Kvadrupel-Zeta   | Kvadrupel-zeta basissett for ekstremt høy nøyaktighet, spesielt i post-HF metoder.                                    | Høynøyaktige beregninger                   |
| **LANL2DZ**      | ECP              | Bruker effektive kjernpotensialer (ECP) for tunge atomer, med dobbelt-zeta basissett for valenselektroner.            | Tunge elementer, overgangsmetaller         |
| **AUG-cc-pVDZ**  | Augmentert Basis | cc-pVDZ med diffuse funksjoner for å fange opp langdistanse-interaksjoner, bedre behandling av svakt bundne systemer. | Svake interaksjoner, anioner, eksiterte tilstander |
| **AUG-cc-pVTZ**  | Augmentert Basis | cc-pVTZ med diffuse funksjoner for mer nøyaktig behandling av eksiterte tilstander og anioner.                        | Svakt bundne systemer, van der Waals interaksjoner |


# Regne ut egenskaper til molekylet

Etter at HF-beregningen er ferdig, kan vi be PySCF om å regne ut ulike egenskaper. Her viser vi hvordan man regner ut dipolmomentet og Mulliken-populasjoner.


In [4]:
dip = mf.dip_moment()
print("Dipole moment: ", dip)
pop = mf.mulliken_pop()
print("Mulliken population: ", pop)



Dipole moment(X, Y, Z, Debye):  1.51362,  1.12058, -0.79173
Dipole moment:  [ 1.51362423  1.12058132 -0.79173428]
 ** Mulliken pop  **
pop of  0 O 1s            1.95076
pop of  0 O 2s            0.99980
pop of  0 O 3s            0.23044
pop of  0 O 4s            0.60320
pop of  0 O 2px           0.44605
pop of  0 O 2py           0.38618
pop of  0 O 2pz           0.42527
pop of  0 O 3px           0.75065
pop of  0 O 3py           0.66838
pop of  0 O 3pz           0.72018
pop of  0 O 4px           0.49027
pop of  0 O 4py           0.36392
pop of  0 O 4pz           0.43285
pop of  0 O 3dxy          0.00057
pop of  0 O 3dyz          0.00016
pop of  0 O 3dz^2         0.00046
pop of  0 O 3dxz          0.00101
pop of  0 O 3dx2-y2       0.00082
pop of  0 O 4dxy          0.00331
pop of  0 O 4dyz          0.00073
pop of  0 O 4dz^2         0.00249
pop of  0 O 4dxz          0.00572
pop of  0 O 4dx2-y2       0.00427
pop of  0 O 4f-3          0.00007
pop of  0 O 4f-2          0.00012
pop of  0 O 4f-

# Gjøre en DFT-beregning

Nå viser vi hvordan man kan gjøre en Kohn-Sham DFT-beregning i PySCF. Dette er ganske likt som Hartree-Fock-beregningen. Vi regner på koffein-molekylet.

Her er en tabell med XC-funksjonaler du kan prøve:

| **Funksjonal**            | **Definisjonsstreng** | **Beskrivelse**                                                                                          |
|---------------------------|-----------------------|----------------------------------------------------------------------------------------------------------|
| **LDA (Local Density Approximation)**        | `'lda'`               | Lokal tetthets-tilnærming. XC-funksjonalen avhenger bare av elektron-tettheten i hvert punkt.               |
| **PBE (Perdew-Burke-Ernzerhof)**  | `'pbe'`               | Generalisert gradient-tilnærming (GGA). Inkluderer gradienten av tettheten for mer nøyaktige beregninger.   |
| **BLYP (Becke-Lee-Yang-Parr)**     | `'blyp'`              | Kombinasjon av Becke's utvekslingsfunksjonal og LYP's korrelasjonsfunksjonal. GGA-nivå funksjonal.           |
| **BP86**                   | `'bp86'`              | Kombinasjon av Becke’s 1988 utvekslingsfunksjonal og Perdew's 1986 korrelasjonsfunksjonal.                  |
| **B3LYP**                  | `'b3lyp'`             | Hybrid GGA funksjonal som kombinerer BLYP med en andel av Hartree-Fock utvekslingsenergi.                   |
| **HSE (Heyd-Scuseria-Ernzerhof)**  | `'hse'`               | Hybrid GGA funksjonal med skjermet utveksling for forbedret behandling av systemer med lange rekkevidde.     |
| **M06**                    | `'m06'`               | Hybrid meta-GGA funksjonal, som inkluderer tetthetsgradienter og kinetisk energitetthet.                    |
| **TPSS (Tao-Perdew-Staroverov-Scuseria)** | `'tpss'`              | Meta-GGA funksjonal som tar hensyn til både elektron-tetthet og kinetisk energitetthet.                     |
| **SCAN**                   | `'scan'`              | Meta-GGA funksjonal som bruker strengt korrelerte tettheter for høyere nøyaktighet i komplekse systemer.    |
| **CAM-B3LYP**              | `'cam-b3lyp'`         | Hybrid funksjonal med lang-rekkevidde korreksjon, basert på B3LYP, men med tilleggsparameter for screening. |
| **PBE0**                   | `'pbe0'`              | Hybridversjon av PBE, som inkluderer en andel av Hartree-Fock utvekslingsenergi.                            |


In [20]:

mf_dft = scf.RKS(mol)

mf_dft.xc = 'b3lyp' # Velg en DFT-funksjonal

energy = mf_dft.kernel()
print("Energy: ", energy)

converged SCF energy = -76.4597337659895
Energy:  -76.4597337659895


# Geometrioptimering

Her viser vi hvordan vi kan gjøre en geometrioptimering. Da prøver PySCF å finne den geometrien som gir lavest energy med modellen og basissettet vi har valgt.


In [22]:
cc_calculation = cc.CCSD(mf)    # Initiate the CCSD calculation with the HF results
ccsd_energy, t1, t2 = cc_calculation.kernel()  # Run the CCSD calculation and get energy and amplitudes

print(f'CCSD total energy: {ccsd_energy:.6f} Ha')

E(CCSD) = -76.33766416831095  E_corr = -0.2815755024872018
CCSD total energy: -0.281576 Ha


In [23]:
from pyscf.geomopt.geometric_solver import optimize
mol_eq = optimize(cc_calculation, maxsteps=100)
print(mol_eq.tostring())


                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********      .))))))  
          ##      .%%%%%%/      (%%%%%%,                  ,******      /)))))  
        %%%%%%      .%%%%%%#      *%%%%%%,    ,/////.       ******      )))))) 
      #%      %%      .%%%%%%/      *%%


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.277400   0.892900   0.254400    0.000000  0.000000  0.000000
   H   0.606800  -0.238300  -0.716900    0.000000  0.000000  0.000000
converged SCF energy = -76.0560886658246
E(CCSD_Scanner) = -76.33766416601958  E_corr = -0.2815755001950165
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0113101600    -0.0083738831     0.0059156588
1 H     0.0032671752     0.0123881446     0.0040838637
2 H     0.0080429848    -0.0040142615    -0.0099995225
----------------------------------------------
cycle 1: E = -76.337664166  dE = -76.3377  norm(grad) = 0.0243854


Step    0 : Gradient = 1.408e-02/1.527e-02 (rms/max) Energy = -76.3376641660
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.60000e-01 5.35840e-01 5.35841e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.002326   0.001722  -0.001216    0.002326  0.001722 -0.001216
   H   0.279075   0.882292   0.246639    0.001675 -0.010608 -0.007761
   H   0.602799  -0.229414  -0.707923   -0.004001  0.008886  0.008977

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -76.0571789479845
E(CCSD_Scanner) = -76.33800214112939  E_corr = -0.2808231931449054
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0009864303    -0.0007303084     0.0005160194
1 H     0.0006355244    -0.0001235651    -0.0006776431
2 H     0.0003509059     0.0008538735     0.0001616237
----------------------------------------------
cycle 2: E = -76.3380021411  dE = -0.000337975  norm(grad) = 0.00187867


Step    1 : Displace = 1.097e-02/1.325e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 1.085e-03/1.331e-03 (rms/max) E (change) = -76.3380021411 (-3.380e-04) Quality = 0.985
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.65419e-01 5.33961e-01 5.35840e-01



Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.004244   0.003142  -0.002220    0.001918  0.001420 -0.001003
   H   0.277516   0.883643   0.248910   -0.001559  0.001350  0.002271
   H   0.602440  -0.232185  -0.709190   -0.000359 -0.002771 -0.001268

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -76.0572439782811
E(CCSD_Scanner) = -76.33801061733425  E_corr = -0.2807666390531512
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O    -0.0000302027    -0.0000223440     0.0000158079
1 H     0.0000174812     0.0000029776    -0.0000149340
2 H     0.0000127215     0.0000193664    -0.0000008739
----------------------------------------------
cycle 3: E = -76.3380106173  dE = -8.4762e-06  norm(grad) = 5.23114e-05


Step    2 : Displace = 2.917e-03/3.068e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 3.020e-05/4.076e-05 (rms/max) E (change) = -76.3380106173 (-8.476e-06) Quality = 1.014
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.61246e-01 5.34166e-01 5.35840e-01



Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.004295   0.003180  -0.002247    0.000051  0.000038 -0.000027
   H   0.277476   0.883672   0.248965   -0.000040  0.000030  0.000055
   H   0.602429  -0.232253  -0.709219   -0.000011 -0.000068 -0.000028
converged SCF energy = -76.0572458100934
E(CCSD_Scanner) = -76.33801072417768  E_corr = -0.2807649140843016
--------------- CCSD_Scanner gradients ---------------
         x                y                z
0 O     0.0000004450     0.0000003286    -0.0000002334
1 H    -0.0000001590    -0.0000003813    -0.0000000700
2 H    -0.0000002861     0.0000000527     0.0000003034
----------------------------------------------
cycle 4: E = -76.3380107242  dE = -1.06843e-07  norm(grad) = 8.44234e-07


Step    3 : Displace = 7.274e-05/7.442e-05 (rms/max) Trust = 2.000e-01 (+) Grad = 4.874e-07/6.004e-07 (rms/max) E (change) = -76.3380107242 (-1.068e-07) Quality = 19.869
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    
Time elapsed since start of run_optimizer: 8.461 seconds


O           0.00430        0.00318       -0.00225
H           0.27748        0.88367        0.24897
H           0.60243       -0.23225       -0.70922


# Beregne vibrasjonelle frekvenser

Som eksempel på hva PySCF kan gjøre viser vi hvordan man beregner vibrasjonelle frekvenser.

Her bruker vi den optimerte geometrien fra forrige beregning. Merk at modellen vår ikke er veldig nøyaktig, så de vibrasjonelle frekvensene er heller ikke veldig nøyaktige.


In [24]:
cc_calculation.mol = mol_eq
cc_calculation.kernel()


E(CCSD) = -76.33766416831095  E_corr = -0.2815755024872018


(-0.28157550248720176,
 array([[ 1.35907225e-04, -1.74137886e-10, -4.50375212e-10,
         -1.26510346e-04,  4.74907287e-05, -3.24490211e-17,
          7.30977679e-05,  2.99855975e-10,  1.50039434e-17,
         -1.26066185e-16, -3.39955675e-04, -7.38579162e-10,
          4.25364368e-09,  2.45000858e-04, -8.27563623e-17,
          1.36655848e-17, -6.30652908e-10,  7.05555612e-05,
          4.07901812e-05, -1.00789746e-05,  4.84576783e-12,
          1.58459165e-17,  6.76832301e-06,  3.46482324e-17,
          2.65722793e-11,  6.31580684e-17, -4.14831075e-10,
         -2.03811972e-05, -1.87642408e-17,  1.30175350e-16,
          2.00478528e-10,  3.51537493e-05, -3.16574434e-17,
          8.41011261e-05, -5.59151552e-10, -5.96356789e-17,
         -1.06951184e-09,  2.70741320e-04,  2.43959328e-04,
         -3.50135770e-10, -2.03621579e-17,  9.24141962e-05,
         -2.47573630e-18,  1.30729685e-17, -1.52023257e-04,
          4.34796970e-17, -7.60606330e-09, -1.13543540e-04,
         -6.03180

In [25]:

from pyscf import eph
myeph = eph.EPH(cc_calculation)

mat, omega = myeph.kernel()

au_to_cmm1 = 219474.63 
print('Vibrasjonelle frekvenser i invers cm:')
print(omega * au_to_cmm1)

TypeError: EPH only supports RHF, UHF, RKS and UKS

In [14]:
mat

array([], shape=(0, 58, 58), dtype=float64)